In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.co

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'company homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [26]:

def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links

In [ ]:
#select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'resume page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'proficiencies page',
   'url': 'https://edwarddonner.com/proficient/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'nebula page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'social profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'social profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'social profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [ ]:
select_relevant_links("https://huggingface.co")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [27]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [ ]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

In [28]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [29]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [30]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [31]:
create_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face: The AI Community Building the Future

---

## Who We Are

Hugging Face is the premier collaboration platform for the global machine learning (ML) community. We bring together engineers, scientists, and AI enthusiasts to share, explore, and build with open-source machine learning models, datasets, and applications. Our mission is to empower the next generation of AI innovators to create an open and ethical AI future, driven by community collaboration and transparency.

---

## What We Offer

- **Hugging Face Hub:** A central place hosting over **2 million models** and **500,000+ datasets** for diverse AI research and applications.
- **Spaces:** Interactive ML applications and demos in text, image, video, audio, and even 3D modalities where users can explore and contribute.
- **Open Source Stack:** A comprehensive and fast-growing open-source ecosystem that accelerates ML development.
- **Community Collaboration:** Tools and infrastructure designed to help users share work, build ML portfolios, and collaborate at scale.
- **Enterprise Solutions:** Customized AI services designed for businesses looking to leverage the power of open machine learning.

---

## Our Community

Hugging Face nurtures a vibrant and rapidly growing community of ML practitioners. Our users range from:
- Researchers pushing the boundary of AI.
- Developers building innovative AI-powered applications.
- Data scientists sharing high-quality datasets.
- End users exploring AI capabilities through accessible apps.

This diverse ecosystem fuels innovation while fostering openness and ethical AI practices.

---

## Why Choose Hugging Face?

- Access to an unparalleled collection of high-quality pre-trained machine learning models.
- Collaborate on projects in any modality — whether text, images, videos, audio, or 3D.
- Build and showcase your AI skills and portfolio with seamless tools.
- Leverage the fastest-growing open-source ML platform to accelerate development.
- Join a supportive, inclusive, and forward-thinking community dedicated to building the future of AI.

---

## Careers at Hugging Face

Join a forward-looking company passionate about democratizing AI! We seek talented individuals excited to:
- Advance open-source machine learning.
- Collaborate across interdisciplinary teams.
- Contribute to creating ethical, state-of-the-art AI products and services.
- Enjoy a culture that encourages learning, innovation, and community impact.

Whether you are an engineering expert, researcher, community manager, or enterprise professional, Hugging Face offers an inspiring environment to grow your career and influence the AI landscape.

---

## Explore More

- Discover AI applications and trending ML models.
- Host your own public models, datasets, or apps.
- Access comprehensive documentation and tutorials.
- Learn how our enterprise solutions can boost your AI initiatives.

Sign up today and be part of the AI revolution!

---

### Contact & Links

**Website:** [huggingface.co](https://huggingface.co)  
**Community Hub:** Collaborate and share your ML projects  
**Enterprise:** Tailored AI solutions for your business  
**Careers:** Explore open positions and join the team  

---

*Hugging Face — Empowering the AI community to build a better future, together.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [32]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [33]:
stream_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face  
*The AI community building the future.*

---

## About Hugging Face

Hugging Face is a pioneering collaboration platform at the heart of the AI revolution. It serves as a global hub where machine learning practitioners, researchers, and developers can share, explore, and develop state-of-the-art machine learning models, datasets, and applications. By fostering an open and ethical AI ecosystem, Hugging Face empowers the next generation of AI engineers, scientists, and enthusiasts to advance the boundaries of artificial intelligence.

---

## What Hugging Face Offers

- **Model and Dataset Hub:** Access and browse over 2 million models and 500,000+ datasets across modalities including text, image, video, audio, and 3D data.
- **Spaces:** A collaborative environment allowing users to create and share AI-powered applications and demos, showcasing real-time use cases like text-to-speech, image generation, and video synthesis.
- **Open-Source Stack:** Accelerate your machine learning projects using Hugging Face's extensive open-source libraries, tools, and frameworks.
- **Community:** Connect with a fast-growing, global community passionate about machine learning, sharing knowledge via forums, blogs, and collaborative projects.
- **Enterprise Solutions:** Tailored AI services designed to integrate and scale machine learning capabilities within organizations.

---

## Company Culture

Hugging Face thrives on openness, collaboration, and innovation. Its culture is rooted in:

- **Community-Driven Development:** Encouraging open source contributions and sharing to build a more ethical and transparent AI future.
- **Inclusivity & Learning:** Supporting continuous learning, skill development, and knowledge exchange among AI professionals and newcomers alike.
- **Cutting-Edge Research:** A dedicated science team at the forefront of AI technology exploring novel ML methods, including NLP, audio, computer vision, reinforcement learning, and ethics.
- **Passion for Impact:** Focused on real-world applications and ethical AI practices that drive meaningful change and democratize AI technology worldwide.

---

## Customers and Users

Hugging Face serves a diverse range of users, including:

- **Machine Learning Engineers & Data Scientists:** Utilizing the platform for experimentation, model development, and building AI-powered applications.
- **Researchers & Academics:** Leveraging open datasets, benchmarks, and community insights to push AI research frontiers.
- **Enterprises & Developers:** Adopting Hugging Face’s APIs, managed services, and customized solutions to accelerate AI deployment and innovation.
- **AI Enthusiasts & Students:** Engaging with the community to learn, contribute, and showcase AI projects on a global stage.

---

## Career Opportunities

Hugging Face is looking for talented individuals passionate about AI and open source who want to make a profound impact in the industry. The company offers ample opportunities across engineering, research, product, and community roles with a culture that fosters growth, creativity, and collaboration.

Explore current openings and join a mission-driven team that values diversity, continuous learning, and cutting-edge innovation.  

*Visit their [Careers page](https://huggingface.co/careers) to learn more.*

---

## In the News & Insights

Hugging Face regularly publishes thought-provoking articles and research through its vibrant blog covering topics such as:

- Advances in natural language processing and reinforcement learning
- Benchmarks and leaderboards for cutting-edge AI tasks  
- Ethical considerations and frameworks in AI development  
- Breakthroughs in AI applications like autonomous driving, voice agents, and computer vision  

Stay connected with the latest developments to deepen your understanding of AI's evolving landscape.

---

## Brand Colors and Assets

- Signature Colors:  
  - Bright Yellow (#FFD21E)  
  - Orange (#FF9D00)  
  - Neutral Gray (#6B7280)

- Logos and design assets are available for download to maintain brand consistency.

---

## Connect with Hugging Face

- **Website:** [huggingface.co](https://huggingface.co)  
- **GitHub:** [github.com/huggingface](https://github.com/huggingface)  
- **Twitter:** [@huggingface](https://twitter.com/huggingface)  
- **LinkedIn:** [Hugging Face](https://linkedin.com/company/huggingface)  
- **Discord Community:** Interactive support and collaboration  

---

**Hugging Face** — Empowering the AI community to create an open, collaborative, and innovative future for machine learning.  
Join the revolution today!